In [1]:
import pandas as pd
import numpy as np
import csv
import glob
import os
from talib.abstract import *
import talib

# 사용 할 보조지표 (추후 추가 가능)

   - ### 주의점: 보조지표는 일자별로 다르게 해야함! 한꺼번에 모든 날짜를 해버리면 안 됨


     - BBANDS  :             Bollinger Bands
     - ADX      :            Average Directional Movement Index
     - RSI      :            Relative Strength Index

## 합칠 numpy data 불러오기

In [2]:
samsung_stoke_data_base = r'D:\dataset\stoke\samsung\data'
samsung_stoke_label_base = r'D:\dataset\stoke\samsung\label'
oil_data_base = r'D:\dataset\stoke\oil\data'
oil_label_base = r'D:\dataset\stoke\oil\label'
ex_bio_data_base = r'D:\dataset\stoke\ex_bio\data'
ex_bio_label_base = r'D:\dataset\stoke\ex_bio\label'
usecols = ['time', 'price', 'sell', 'buy', 'trading_volum']

## 여러 주식 data 요소들을 하나의 numpy배열로 묶기

In [72]:
class Data_join:
    path = ''
    sequence_length = 0
    data = np.array([])
    label = np.array([])
    
    #초기값 설정
    def __init__(self, value1, value2):
        self.sequence_length = value1
        self.path = value2

    
    #데이터 판다스로 받음
    def datagen(self):
        data = pd.read_csv(base_data + self.path + '.csv')
        open1 = data['open'].values
        open1 = open1.astype('float32')
        volume = data['volume'].values.astype('float32')
        high = data['high'].values.astype('float32')
        low = data['low'].values.astype('float32')
        close = data['close'].values.astype('float32') 
        time = data['when'].values
     
    #보조지표 만들기
    def auxiliary(self):
        inputs = {
            'open': self.open1.astype('double'),
            'high': self.high.astype('double'),
            'low' : self.low.astype('double'),
            'close': self.close.values.astype('double'),
            'volume': self.volume.astype('double')
        }
        #날짜가 다르면 끊고 보조지표를 다시 만듦
        for k in range(len(self.time)):
            if self.time[k][:8] != self.time[k+1][:8]:
                
        
        #데이터 정규화
        #open
        mean_open = open1.mean(axis =0)
        open1 -= mean_open
        std_open = open1.std(axis = 0)
        open1 /= std_open
        #volumn
        mean_volumn = volumn.mean(axis = 0)
        volumn -= mean_volumn
        std_volumn = volumn.std(axis = 0)
        volumn /= std_volumn
        #high
        mean_high = high.mean(axis = 0)
        high -= mean_high
        std_high = high.std(axis = 0)
        high /= std_high
        #close
        mean_close = close.mean(axis = 0)
        close -= mean_close
        std_close = close.std(axis = 0)
        close /= std_close
        #low
        mean_low = low.mean(axis = 0)
        low -= mean_low
        std_low = low.std(axis = 0)
        low /= std_low
        
        #두 데이터 합치기
        open1 = open1[:, np.newaxis]
        volumn = volumn[:, np.newaxis]
        high = high[:, np.newaxis]
        close = close[:, np.newaxis]
        low = low[:, np.newaxis]
        real_data = np.concatenate((open1, close, high, low, volumn), axis = 1)
        
        #window 만들기  
        result = []
        for index in range(len(real_data) - self.sequence_length):
            result.append(real_data[index: index + self.sequence_length])
        
        new_data = np.array(result)
        #new_data = new_data[:, :, :, np.newaxis]
        self.data = new_data
        del result

    def labelgen(self):
        label = np.load(base_label + self.path+'.npy')
        label = label[self.sequence_length - 1:]
        self.label = label

In [8]:
data = pd.read_csv( r'D:\dataset\stoke\new_total\data'+ '\\samsung.csv')

In [10]:
time = data['when'].values

In [25]:
for k in range(len(time) - 1):
    if time[k][:8] != time[k+1][:8]:
        print('a')

a
a
a
a
a
a
a


In [20]:
time[0][:8] == time[200][:8] 

False

In [13]:
print(data)

               when   open   high    low  close   volume
0     20201102 1213  56900  56900  56800  56900     5655
1     20201102 1214  56900  57000  56800  57000    82444
2     20201102 1215  56900  57000  56900  57000     4728
3     20201102 1216  57000  57000  56900  57000     8625
4     20201102 1217  57000  57000  56900  56900     5476
...             ...    ...    ...    ...    ...      ...
2851  20201111 1517  61100  61200  61100  61200   179123
2852  20201111 1518  61100  61200  61100  61100    81822
2853  20201111 1519  61100  61200  61100  61100    76017
2854  20201111 1520  61100  61300  61100  61200    54864
2855  20201111 1530  61300  61300  61300  61300  1251652

[2856 rows x 6 columns]


In [40]:
data_df = pd.read_csv( r'D:\dataset\stoke\new_total\data'+ '\\samsung.csv')

In [42]:
inputs = {
    'open': data_df['open'].values.astype('double'),
    'high': data_df['high'].values.astype('double'),
    'low' : data_df['low'].values.astype('double'),
    'close': data_df['close'].values.astype('double'),
    'volume': data_df['volume'].values.astype('double')
}

In [43]:
inputs

{'open': array([56900., 56900., 56900., ..., 61100., 61100., 61300.]),
 'high': array([56900., 57000., 57000., ..., 61200., 61300., 61300.]),
 'low': array([56800., 56800., 56900., ..., 61100., 61100., 61300.]),
 'close': array([56900., 57000., 57000., ..., 61100., 61200., 61300.]),
 'volume': array([   5655.,   82444.,    4728., ...,   76017.,   54864., 1251652.])}

In [44]:
SMA = SMA(inputs, timeperiod=5)

In [49]:
len(SMA[4:])

2852

In [46]:
RSI = RSI(inputs, timeperiod = 5)

In [47]:
len(RSI)

2856

In [53]:
ADX = ADX(inputs, timeperiod = 5)

In [54]:
BBANDS = BBANDS(inputs, timeperiod = 5)

In [64]:
ADX = np.array(ADX)

In [65]:
RSI = np.array(RSI)

In [67]:
RSI = RSI[:, np.newaxis]
ADX = ADX[: , np.newaxis]

In [70]:
pre_data.shape

(2851, 5, 5)

## 기존 데이터와 보조지표 합치기

In [69]:
post_data = np.concatenate((pre_data, RSI, ADX), axis = 1)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)